# Custom Researcher Workflow
This notebook demonstrates how to create a custom workflow using the `AgentWorkflow` class.

This example extract contents from 3 urls, extract insights from the contents, review the insights, and provide a detailed report based on the insights.
- "https://www.anthropic.com/engineering/building-effective-agents",
- "https://www.ibm.com/think/topics/agentic-workflows",
- "https://aws.amazon.com/blogs/hpc/building-an-ai-simulation-assistant-with-agentic-workflows/"

# Imports

In [1]:
import asyncio
from dotenv import load_dotenv
from typing import Any, Dict, List
import requests
from bs4 import BeautifulSoup

from saw.workflow import AgentWorkflow
from saw.core.model_interface import amodel_call

In [2]:
# Load environment variables
load_dotenv("../docker/.env")

True

# Define Custom Researcher Function
Define a custom function that will be used by the `AgentWorkflow` to research content.


In [3]:
async def custom_researcher(urls: List[str], **params: Dict[str, Any]) -> str:
    async def fetch_content(url: str) -> str:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()

    # Remove any unexpected keys from params
    valid_keys = {"provider", "model", "system_prompt"}
    params = {k: v for k, v in params.items() if k in valid_keys}

    contents = await asyncio.gather(*[fetch_content(url) for url in urls])
    combined_content = "\n\n".join(contents)

    insights = []
    for content in contents:
        print(f"Extracting insights from: {content}")
        insight = await amodel_call(
            prompt=f"Extract essential insights from the following content:\n{content}",
            system_prompt="You are an insightful researcher.",
            **params
        )
        insights.append(insight)

    print(f"Insights: {insights}")

    principal_researcher_approval = await amodel_call(
        prompt=f"Review the following insights and indicate if more details are needed:\n{insights}",
        system_prompt="You are a principal researcher.",
        **params
    )

    print(f"Principal Researcher Approval: {principal_researcher_approval}")

    if principal_researcher_approval and "more details" in principal_researcher_approval.lower():
        insights = []
        for content in contents:
            insight = await amodel_call(
                prompt=f"Extract more detailed insights from the following content:\n{content}",
                system_prompt="You are an insightful researcher.",
                **params
            )
            insights.append(insight)

    tech_lead_report = await amodel_call(
        prompt=f"Provide a detailed report based on the following insights:\n{insights}",
        system_prompt="You are a tech lead.",
        **params
    )

    print(f"Tech Lead Report: {tech_lead_report}")

    return tech_lead_report

# Initialize Agent Workflow

In [4]:
# Initialize the `AgentWorkflow` with the custom researcher function.
agent = AgentWorkflow(operation="custom", custom_function=custom_researcher)

In [5]:
# Define Query and Prompts
urls = [
    "https://www.anthropic.com/engineering/building-effective-agents",
    "https://www.ibm.com/think/topics/agentic-workflows",
    "https://aws.amazon.com/blogs/hpc/building-an-ai-simulation-assistant-with-agentic-workflows/"
]

params = {
    "provider": "google",
    "model": "gemini-2.0-flash",
}

In [6]:
# Execute the workflow
result = await agent.execute(urls=urls, async_mode=True, **params)

Extracting insights from: Just a moment...Enable JavaScript and cookies to continue
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=286 prompt_token_count=25 total_token_count=311
Extracting insights from: 








What are Agentic Workflows? | IBM













































































                        



  
    What are agentic workflows?






    


                    


























                                    Artificial Intelligence
                                



                                    IT automation
                                






                    



  
    7 March 2025






    


                












                Link copied
            



































In [7]:
print(result)

Okay, based on the provided insights, here's a detailed report on the implications of JavaScript/Cookies, Agentic Workflows, and a specific AWS implementation of an AI Simulation Assistant using agentic workflows. This report aims to provide a comprehensive understanding of each area and how they intersect.

## Report: JavaScript/Cookies, Agentic Workflows, and AI Simulation Assistants

**Executive Summary:**

This report explores three distinct but related areas: the foundational web technologies of JavaScript and cookies, the emerging field of Agentic Workflows powered by AI, and a practical application of Agentic Workflows in the form of an AWS-based AI Simulation Assistant. We will examine the characteristics, benefits, challenges, and implications of each topic, ultimately connecting them through the common thread of enhanced automation and dynamic adaptation.

**I. JavaScript and Cookies: The Foundation of Dynamic Websites**

*   **Core Functionality:**  The "Just a moment...Enab